## Removing words for merging food items

> input: manually seletced words to remove

> output: possiblely shorteded names of food items

In [1]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712


In [2]:
import myfunctions as f
f.today()

'20180305'

In [3]:
## output from food synset entities
file_name = "food_entities__20180208.p"
df_retrived = f.retrive_file(file_name)

retrived: food_entities__20180208.p


In [4]:
print df_retrived.shape
df_retrived.head(2)

(92298, 11)


,index,food_name,locs,synonyms,menu_names,restaurant,review,review.clean_text,review.restaurant,review.text,review.image
0,food_0,-PRON- ann,"[deliveroo/food/68155, deliveroo/food/AV062oM7...",[em ann],"[(Em Ann M., 3)]","[delivery_3748, delivery_3749, delivery_3750]",[burppleinitial/review/vGui2G8q],[maggie mee goreng from the rooftop at screeni...,"[(burpple_12581, 1)]",[Maggi Mee Goreng from The Rooftop at Screenin...,[vGui2G8q]
1,food_1,-PRON- bai vege,"[foodpanda/menu_item/AV44Ejd8W4PLiMnQOaiu, foo...",[you bai vege],"[(H6. You Bai Vege, 2)]","[delivery_221, delivery_222]",NaN,NaN,NaN,NaN,NaN


In [5]:
food_names = df_retrived["food_name"].tolist()

In [6]:
# count frequency of occurance of the unigram tokens
tokens = [s.split() for s in food_names]
from collections import Counter
c = Counter([item for sublist in tokens for item in sublist])
count_tokens = sorted(c.items(), key=lambda x: -x[1])

In [15]:
count_tokens[:5]

[(u'with', 16061),
 (u'chicken', 11072),
 (u'fry', 8164),
 (u'rice', 6989),
 (u'and', 6657)]

In [16]:
# all tokens
unique_tokens = [s[0] for s in count_tokens]
print "number of uinique tokens: %d"%len(unique_tokens)

number of uinique tokens: 14644


## Checking the words to be removed

In [34]:
# import file with possible error
file_name = "clean_food_remove_words.csv"
import pandas as pd
rmv = pd.read_csv(file_name, header=None)[0].tolist()
print "number of words to remove: %d"%len(rmv) 

number of words to remove: 400


In [72]:
# select only unigram rmv
rmv_uni = [s for s in rmv if len(s.split())==1]
in_token = [s for s in rmv_uni if s in unique_tokens]
# lemma form to match against cleaned names
out_token = [s for s in rmv_uni if s not in unique_tokens]
import spacy
nlp = spacy.load('en_core_web_sm')
def lemma_name(s):
    return " ".join([token.lemma_ for token in nlp(unicode(s, "utf-8"))])
out_token_lemma = [lemma_name(f.clean_name_v4(s)) for s in out_token]
in_token_lemma = [s for s in out_token_lemma if s in unique_tokens]
# all uni-gram tokens sorted by frequency of occurance in food names
rmv_uni = in_token + in_token_lemma
rmv_uni = [s for s in unique_tokens if s in rmv_uni]
print "number of unigram words possible to remove: %d"%len(rmv_uni)

number of unigram words possible to remove: 248


In [82]:
rmv_uni[:7]

[u'with', u'and', u'sauce', u'set', u'spicy', u'grill', u'slice']

In [78]:
rmv_multi = [s for s in rmv_new if len(s.split())>1]
# lemma form to match against cleaned names
import spacy
nlp = spacy.load('en_core_web_sm')
def lemma_name(s):
    return " ".join([token.lemma_ for token in nlp(unicode(s, "utf-8"))])
rmv_multi_lemma = [lemma_name(f.clean_name_v4(s)) for s in rmv_multi]
rmv_multi = sorted(list(set(rmv_multi_lemma + rmv_multi)))
# check if they are contained in some food names
in_multi_token = []
for s1 in sorted(list(set(food_names))):
    s1 = " " + s1 + " "
    for s2 in rmv_multi:
        if " " + s2 + " " in s1:
            in_multi_token.append(s2)
# check if they are composed of single removavble tokens 
rmv_multi_tmp = set(in_multi_token)
rmv_multi = []
for s in rmv_multi_tmp:
    if len(set(s.split()) | set(rmv_uni)) > len(set(rmv_uni)):
        rmv_multi.append(s)
print "number of multigram words to be removed: %d"%len(rmv_multi)

number of multigram words to be removed: 46


In [84]:
rmv_multi[:6]

[u'duo of', u'the ugly', u'cup of', 'stir fried', u'piece of', u'monday blue']

In [79]:
wrongly_rmv = ["stir fry", 'stir fried']
rmv = rmv_uni + rmv_multi
rmv = [s for s in rmv if s not in wrongly_rmv]
print "number of words to remove: %d"%len(rmv) 

number of words to remove: 292


In [89]:
# save the file back to words to remove
file_name = "words_to_remove_" + f.today() + ".csv"
import csv
with open(file_name, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    for val in rmv:
        writer.writerow([val])  
print ("retrived: %s" % file_name)

retrived: words_to_remove_20180305.csv


## shorten food name

In [88]:
from copy import deepcopy
df = deepcopy(df_retrived)

In [96]:
# import file, longer words first
file_name = "words_to_remove_20180305.csv"
import pandas as pd
rmv = pd.read_csv(file_name, header=None)[0].tolist()
rmv.sort(key=lambda item: (-len(item), item))
print "number of words to remove: %d"%len(rmv)  

number of words to remove: 292


In [99]:
def shorten_name(name):
    name = " " + name + " "
    for s in rmv:
        if " " + s + " " in name:
            name = name.replace(" " + s + " ", " ")
    return name.strip()

In [100]:
%%time
df["short_name"] = df["food_name"].apply(shorten_name)

CPU times: user 6.4 s, sys: 72 ms, total: 6.47 s
Wall time: 6.39 s


In [101]:
df.head(2)

,index,food_name,locs,synonyms,menu_names,restaurant,review,review.clean_text,review.restaurant,review.text,review.image,short_name
0,food_0,-PRON- ann,"[deliveroo/food/68155, deliveroo/food/AV062oM7...",[em ann],"[(Em Ann M., 3)]","[delivery_3748, delivery_3749, delivery_3750]",[burppleinitial/review/vGui2G8q],[maggie mee goreng from the rooftop at screeni...,"[(burpple_12581, 1)]",[Maggi Mee Goreng from The Rooftop at Screenin...,[vGui2G8q],-PRON- ann
1,food_1,-PRON- bai vege,"[foodpanda/menu_item/AV44Ejd8W4PLiMnQOaiu, foo...",[you bai vege],"[(H6. You Bai Vege, 2)]","[delivery_221, delivery_222]",NaN,NaN,NaN,NaN,NaN,-PRON- bai vege


In [102]:
df["short_name"].nunique()

75908

In [103]:
df.shape

(92298, 12)

=> name shortening works, but need to check the merged records